In [1]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/alpaco/anaconda3/envs/unsloth-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info


🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:

import torch
torch.set_default_device("cuda")  # 새 텐서 기본 디바이스 CUDA

from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForLanguageModeling
import torch
from datasets import load_dataset
from transformers import TrainingArguments

ds = load_dataset("ChuGyouk/PubMedQA-test-Ko")
df = ds["test"].to_pandas().dropna(subset=["QUESTION", "LONG_ANSWER"])
df = df[df["QUESTION"].str.len() > 5]
df = df[df["LONG_ANSWER"].str.len() > 5]

from datasets import Dataset
df["text"] = "질문: " + df["QUESTION"] + "\n답변: " + df["LONG_ANSWER"]
dataset = Dataset.from_pandas(df[["text"]])
split = dataset.train_test_split(test_size=0.2, seed=42)
train_ds, valid_ds = split["train"], split["test"]


In [21]:
import torch
from datasets import load_dataset
from transformers import TrainingArguments

max_seq_length = 512
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
    device_map="auto",
)

# 학습 모드로 전환
model = FastLanguageModel.for_training(model)

==((====))==  Unsloth 2025.10.9: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 2. Max memory: 23.691 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [22]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # GPU 0번만 사용하도록 고정

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16,
    lora_dropout=0.0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print(f" 모델 로드 완료 ({device})")

 모델 로드 완료 (cuda)


In [ ]:
dl = trainer.get_train_dataloader()
batch = next(iter(dl))
batch = {k: v.to("cuda:0") for k, v in batch.items() if hasattr(v, "to")}
with torch.no_grad():
    _ = model(**batch)
print("✅ 1-batch GPU forward OK")

In [ ]:
max_seq_length = 512

# 1) 모델 로드: device_map=None로 두고 명시적으로 CUDA:0로 이동
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
    device_map=None,     # auto 금지 (혼선 방지)
)
model = model.to("cuda:0")
model = FastLanguageModel.for_training(model)

# 2) CUDA 강제 collator (배치가 CPU에 남지 않도록 보장)
base_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
def cuda_collator(features):
    batch = base_collator(features)
    return {k: (v.cuda(non_blocking=True) if torch.is_tensor(v) else v) for k, v in batch.items()}

# 3) TrainingArguments (안정화 옵션 포함)
training_args = TrainingArguments(
    output_dir="outputs",
    per_device_train_batch_size=3,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=60,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=5,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    eval_strategy="steps",
    eval_steps=10,
    save_strategy="no",
    report_to="none",
    remove_unused_columns=False,
    # 안정화 옵션
    dataloader_pin_memory=True,
    dataloader_num_workers=0,
)

# 4) SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
    data_collator=cuda_collator,  # 🔴 중요: CUDA 강제 collator 사용
)

# 5) train() 전에 1-batch sanity check (GPU로 잘 가는지 확인)
dl = trainer.get_train_dataloader()
batch = next(iter(dl))
batch = {k: (v.to("cuda:0") if torch.is_tensor(v) else v) for k, v in batch.items()}
with torch.no_grad():
    _ = model(**batch)
print("✅ 1-batch GPU forward OK")

# 6) 학습
trainer_stats = trainer.train()
print("✅ Fine-tuning complete!")

# 7) Inference 모드 전환 및 저장
FastLanguageModel.for_inference(model)
model.save_pretrained("lora_model_llama3")
tokenizer.save_pretrained("lora_model_llama3")

try:
    model.save_pretrained_merged("lora_model_llama3_merged", tokenizer, save_method="merged_16bit")
    print("✅ 병합 저장 완료")
except Exception as e:
    print("⚠️ 병합 저장 실패:", e)

trainer.save_state()
trainer.save_model("lora_model_llama3_trainer")
print("✅ Trainer 상태 저장 완료")


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_test_ds = valid_ds.map(tokenize_function, batched=True)
# 평가 시에도 같은 collator를 쓸 수 있게 trainer에 이미 지정되어 있음
eval_results = trainer.evaluate(tokenized_test_ds)
print("📊 평가 결과:", eval_results)

# ------------------------ Test (생성) ------------------------
# 병합 모델(16bit) 로드 시 device_map=None + 명시적 CUDA 이동 권장
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="lora_model_llama3_merged",
    max_seq_length=512,
    dtype=torch.float16,  
    load_in_4bit=False,
    device_map=None,
)
model = model.to("cuda:0")
model.eval()

In [25]:
prompt = "배변 시 핸드폰 사용 습관이 직장암에 미치는 영향은?"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=128)
print("💬 모델 응답:", tokenizer.decode(outputs[0], skip_special_tokens=True))

💬 모델 응답: 배변 시 핸드폰 사용 습관이 직장암에 미치는 영향은? 
직장암 환자군과 대조군을 비교하여 연구한 결과, 직장암 환자군의 핸드폰 사용 습관을 분석한 결과, 핸드폰 사용 후 배변 시 핸드폰 사용이 직장암 발병률이 더 높게 나타났습니다. 
직장암 환자군의 핸드폰 사용 습관은 핸드폰 사용 후 배변 시 핸드폰 사용이 직장암 발병률이 더 높게 나타났습니다. 
직장암 환자군의 핸드폰 사용 습
